In [1]:
minmaxInterval = 2500

## open port

In [1]:
!echo 'qwer1234'| sudo -S chmod a+rw /dev/ttyUSB0

[sudo] password for terry-q: 

## BASIC USAGE OF XL320 CLASS

In [2]:
import os, time
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from math import pi as PI
DEG90 = PI/2
PROPORTION_MATRIX = np.array(
    [
        [1,0.1,0.4,0.4, 0,0,0,0, 0,0,0,0],
        [0.1,1,0.4,0.4, 0,0,0,0, 0,0,0,0],
        [0.4,0.4,1,0.1, 0,0,0,0, 0,0,0,0],
        [0.4,0.4,0.1,1, 0,0,0,0, 0,0,0,0],
        
        [0,0,0,0,  1,0.1,0.4,0.4, 0,0,0,0],
        [0,0,0,0,  0.1,1,0.4,0.4,  0,0,0,0],
        [0,0,0,0,  0.4,0.4,1,0.1, 0,0,0,0],
        [0,0,0,0,  0.4,0.4,0.1,1, 0,0,0,0],
        
        [0,0,0,0, 0,0,0,0, 1,0.1,0.4,0.4],
        [0,0,0,0, 0,0,0,0, 0.1,1,0.4,0.4],
        [0,0,0,0, 0,0,0,0, 0.4,0.4,1,0.1],
        [0,0,0,0, 0,0,0,0, 0.4,0.4,0.1,1]
        ])


In [3]:
import sys
sys.path.append(".")
""" FOR DYNAMIXEL """
from model.class_xc330 import xc330
""" FOR MODERN DRIVER """
import roslib; roslib.load_manifest('ur_driver')
import rospy
import actionlib
from control_msgs.msg import *
from trajectory_msgs.msg import *
from sensor_msgs.msg import JointState

from class_vicon_marker import VICON
from class_vicon_base import ViconBase

class Fingers(object):
    def __init__(self,):
        rospy.init_node("DEMO")
        self.JOINT_NAMES = ['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint',
                            'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
        self.client      = None
        
        self.xc          = xc330('SOFTROBOT', _USB_NUM=0)
        
    def init_xc(self,initpos):
        self.xc.connect()
        self.xc.set_operatingmode([4])

        self.xc.set_torque([0])
        time.sleep(2)
        self.xc.set_torque([1])
        print ("TORQUE ON")

        currpos = self.xc.get_currpos()
        minmaxInterval = 3000
        self.xc.set_minmaxpos(currpos-np.ones_like(currpos)*minmaxInterval,currpos+np.ones_like(currpos)*minmaxInterval)

        if (initpos==None).any(): initpos = currpos; print("No initpos Found")
        self.initpos = initpos
        self.xc.set_goalposcluster(initpos,1)

        print ("INITIALIZE POSITION")
        currpos = self.xc.get_currpos()
        print (currpos)

        minmaxInterval = minmaxInterval
        self.xc.set_minmaxpos(currpos-np.ones_like(currpos)*minmaxInterval,currpos+np.ones_like(currpos)*minmaxInterval)
    
    def move_xc(self,pos):
        self.xc.set_goalposcluster(pos,1)
        
    def pmove_xc(self,pos):
        pos = self.initpos + PROPORTION_MATRIX[:12,:12] @ np.array(pos) * np.array([1,-1,-1,1]*3)
        self.xc.set_goalposcluster(pos,1)
        
    def move_arm(self, joints):
        try: 
            q = joints
            g = FollowJointTrajectoryGoal()
            g.trajectory = JointTrajectory()
            g.trajectory.joint_names = self.JOINT_NAMES
            joint_states = rospy.wait_for_message("joint_states", JointState)
            joints_pos   = joint_states.position
            g.trajectory.points = [
                JointTrajectoryPoint(positions=joints_pos, velocities=[0]*6, time_from_start=rospy.Duration(0.0)),
                JointTrajectoryPoint(positions=q, velocities=[0]*6, time_from_start=rospy.Duration(3))]  
            self.client.send_goal(g)
            self.client.wait_for_result()
        except KeyboardInterrupt:
            self.client.cancel_goal()
            raise
        except:
            raise  
         
    def execute_arm(self, joints):
        try:
            self.client = actionlib.SimpleActionClient('follow_joint_trajectory', FollowJointTrajectoryAction)
            print("Waiting for server...")
            self.client.wait_for_server()
            print("Connected to server")
            """ Initialize """
            self.move_arm(joints)
            print("Finish plan")

        except KeyboardInterrupt:
            rospy.signal_shutdown("KeyboardInterrupt")
            raise      


['/home/terry-q/Rilab/sOftrobot/SORO-Dynamixel-python', '', '/home/terry-q/catkin_ws/devel/lib/python3/dist-packages', '/opt/ros/noetic/lib/python3/dist-packages', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '/home/terry-q/.local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '.']


## INSTANTIATE Robot 

In [4]:
robot = Fingers()

[SOFTROBOT] INSTANTIATED AT [b'/dev/ttyUSB0']


In [7]:
init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
robot.execute_arm(init_joint)

Waiting for server...
Connected to server
Finish plan


# Calibrate initpos
반드시 Cap을 벗기고 진행할것

In [6]:
# Upright_joint = np.array([0, -90+45, 90-45, -90, 90, 90])/90*DEG90
# robot.execute_arm(Upright_joint)

In [7]:
# robot.init_xc(initpos=np.array([None]))
# initpos = robot.xc.get_currpos()

In [8]:
# relaxed_pos = initpos

In [9]:
# initpos = np.array([2247, -546,  344, 2629, 1000, 2833, 1801, 4351, 4761, 2223, 1299,
#        1990,  2605, -703, 2454, 1732])
# # initpos[-4:] = relaxed_pos[-4:]
# robot.init_xc(initpos=initpos)

In [10]:
# relaxed_pos

In [11]:
# # 1번은 플러스, 2번 마이너스, 3번 마이너스, 4번 플러스
# pos = initpos + np.array([0,0,0,0, 0,0,0,0, 0,0,0,0, 100,200,200,200]) * np.array([1,-1,-1,1]*4)
# # robot.move_xc(pos)

# pos = pos + PROPORTION_MATRIX @ np.array([0,0,0,0, 0,0,0,0, 0,0,0,0, 600,600,600,600]) * np.array([1,-1,-1,1]*4)

# robot.move_xc(pos)

# robot.xc.get_currpos()

In [12]:
# initpos = np.array([2247, -546,  344, 2629, 1000, 2833, 1801, 4351, 4761, 2223, 1299,
#        1990, 4110,  970, 3106, 3197])
# robot.init_xc(initpos)
# robot.xc.get_currpos()

# pos = initpos + np.array([300]*16) * np.array([1,-1,-1,1]*4)
# robot.move_xc(pos)
# time.sleep(2)

# pos = initpos + PROPORTION_MATRIX @ np.array([1000,0,0,0, 0,0,1000,0, 0,1000,0,0, 0,0,0,1000]) * np.array([1,-1,-1,1]*4)
# robot.move_xc(pos)
# time.sleep(2)


# pos = initpos + PROPORTION_MATRIX @ np.array([0,1000,0,0, 0,0,0,1000, 1000,0,0,0, 0,0,1000,0]) * np.array([1,-1,-1,1]*4)
# robot.move_xc(pos)
# time.sleep(2)

# robot.move_xc(initpos)

# Pick and place

In [8]:
robot.init_xc(np.array([None]))
initpos = robot.xc.get_currpos()
pos = initpos + np.array([1,1,1,1, 1,1,1,1, 1,1,1,1])*1000 * np.array([1,-1,-1,1]*3)
robot.move_xc(pos)

initpos = np.array([2247, -546,  344, 2629, 1000, 2833, 1801, 4351, 4761, 2223, 1299,
       1990])
robot.init_xc(initpos)

robot.xc.get_currpos()

[SOFTROBOT] CONNECT
 SUCCEEDED TO OPEN THE PORT!
 SUCCEEDED TO CHANGE THE BAUDRATE!
COMMUNICATION SUCCESS
[SOFTROBOT] DETECT ID
[SOFTROBOT] [12]MOTORS DETECTED 
[SOFTROBOT] ID_LIST:[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]
TORQUE ON
No initpos Found
INITIALIZE POSITION
[1727  395  996 1884  444 3570 2347 3678 4144 3349 2209  995]
[SOFTROBOT] CONNECT
[PortHandler setup] The port number 0 has same device name... reinitialize port number 0!!
 SUCCEEDED TO OPEN THE PORT!
 SUCCEEDED TO CHANGE THE BAUDRATE!
COMMUNICATION SUCCESS
[SOFTROBOT] DETECT ID
[SOFTROBOT] [12]MOTORS DETECTED 
[SOFTROBOT] ID_LIST:[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]
TORQUE ON
INITIALIZE POSITION
[2230 -513  368 2601  984 2859 1822 4325 4737 2259 1332 1957]


array([2238, -529,  359, 2613,  991, 2848, 1815, 4335, 4748, 2242, 1317,
       1974])

# Pick and Place

In [12]:
# Ungrasp
robot.pmove_xc(np.array([2000,0,0,0]*3))

In [13]:
# Jar
target_down = np.array([-52.15, -76.28, 104.92, -118.90, -89.82, 38.04])/90*DEG90
target_up = np.array([-52.12, -82.53, 89.23, -96.96, -89.76, 37.92])/90*DEG90

locate_down = np.array([5.26, -87.29, 118.61, -121.27, -90.09, 95.48])/90*DEG90
locate_up = np.array([5.30, -94.65, 101.60, -96.90, -90.02, 95.36])/90*DEG90

# approach up
robot.execute_arm(target_up)
time.sleep(2)

# approach down
robot.execute_arm(target_down)
time.sleep(2)

# Grasp
robot.pmove_xc(np.array([0,2500,0,0]*3))

# Pick up
robot.execute_arm(target_up)
time.sleep(2)

robot.execute_arm(locate_up)
time.sleep(2)

robot.execute_arm(locate_down)
time.sleep(2)

# Ungrasp
robot.pmove_xc(np.array([2000,0,0,0]*3))

robot.execute_arm(locate_up)
time.sleep(2)


Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan


In [14]:
# air feshner
target_down = np.array([-33.68, -66.54, 93.37, -117.01, -89.93, 56.48])/90*DEG90
target_up = np.array([-33.65, -72.65, 73.96, -91.50, -89.88, 56.34])/90*DEG90

locate_down = np.array([2.80, -68.79, 97.70, -118.87, -90.07, 92.98])/90*DEG90
locate_up = np.array([2.84, -75.76, 75.24, -89.45, -90.00, 92.82])/90*DEG90

# approach up
robot.execute_arm(target_up)
time.sleep(2)

# approach down
robot.execute_arm(target_down)
time.sleep(2)

# Grasp
robot.pmove_xc(np.array([0,2500,0,0]*3))

# Pick up
robot.execute_arm(target_up)
time.sleep(2)

robot.execute_arm(locate_up)
time.sleep(2)

robot.execute_arm(locate_down)
time.sleep(2)

# Ungrasp
robot.pmove_xc(np.array([2000,0,0,0]*3))

robot.execute_arm(locate_up)
time.sleep(2)

Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan


In [15]:
# tennis ball
target_down = np.array([-17.53, -45.75, 70.21, -114.56, -89.98, 72.60])/90*DEG90
target_up = np.array([-17.52, -51.78, 57.30, -95.62, -89.94, 72.48])/90*DEG90

locate_down = np.array([1.55, -49.04, 76.16, -117.11, -90.05, 91.69])/90*DEG90
locate_up = np.array([1.56, -55.77, 63.78, -97.99, -90.00, 91.59])/90*DEG90


# approach up
robot.execute_arm(target_up)
time.sleep(2)

# approach down
robot.execute_arm(target_down)
time.sleep(2)

# Grasp
robot.pmove_xc(np.array([0,2500,0,0]*3))

# Pick up
robot.execute_arm(target_up)
time.sleep(2)

robot.execute_arm(locate_up)
time.sleep(2)

robot.execute_arm(locate_down)
time.sleep(2)

# Ungrasp
robot.pmove_xc(np.array([2000,0,0,0]*3))

robot.execute_arm(locate_up)
time.sleep(2)

Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan


In [16]:
# water bottle
target_down = np.array([-39.24, -50.22, 59.18, -99.17, -89.84, 50.78])/90*DEG90
target_up = np.array([-39.22, -50.46, 42.64, -82.39, -89.83, 50.67])/90*DEG90

locate_down = np.array([1.48, -40.42, 39.82, -89.37, -89.97, 91.43])/90*DEG90
locate_up = np.array([1.50, -35.58, 11.10, -70.50, -89.98, 91.30])/90*DEG90

# approach up
robot.execute_arm(target_up)
time.sleep(2)

# approach down
robot.execute_arm(target_down)
time.sleep(2)

# Grasp
robot.pmove_xc(np.array([0,2500,0,0]*3))

# Pick up
robot.execute_arm(target_up)
time.sleep(2)

robot.execute_arm(locate_up)
time.sleep(2)

robot.execute_arm(locate_down)
time.sleep(2)

# Ungrasp
robot.pmove_xc(np.array([2000,0,0,0]*3))

robot.execute_arm(locate_up)
time.sleep(2)

Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan


In [17]:
robot.execute_arm(init_joint)

Waiting for server...
Connected to server
Finish plan


# Tennis

In [55]:
# Grasp
robot.pmove_xc(np.array([0,2500,0,0]*3))

In [56]:
# Ungrasp
robot.pmove_xc(np.array([2000,0,0,0]*3))

In [95]:
# robot.execute_arm(Place_up_joint)
# init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
# robot.execute_arm(init_joint)

In [48]:
init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
Target_up_joint = np.array([-23.10, -41.35, 58.77, 107.57])/90*DEG90

Target_down_joint = np.array([-32.72, -67.72, 103.66, -126.13, -89.92, 57.51])/90*DEG90

Place_up_joint = np.array([])/90*DEG90

Place_down_joint = np.array([-38.07,-49.32,77.91,-118.45,-89.65,94.72])/90*DEG90

assert (Target_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()


# robot.execute_arm(init_joint)

# robot.execute_arm(Target_up_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))

# robot.execute_arm(Target_down_joint)

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

# 들고
robot.pmove_xc(np.array([0,2000,3000,1000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# # 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

# robot.execute_arm(Target_up_joint)
# robot.execute_arm(Place_up_joint)
# robot.execute_arm(Place_down_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))
# robot.execute_arm(Place_up_joint)
# robot.execute_arm(init_joint)

# Tumbler

In [43]:
robot.pmove_xc(np.array([0,0,0,0]*4))

In [44]:
# robot.execute_arm(Place_up_joint)
# init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
# robot.execute_arm(init_joint)

In [45]:
init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
Target_up_joint = np.array([-34.38,-90.85,93.59,-92.60,-89.61,98.27])/90*DEG90
Target_down_joint = np.array([-34.42,-87.67,107.73,-109.93,-89.65,98.36])/90*DEG90
Place_up_joint = np.array([-25.41,-66.39,63.93,-87.40,-89.59,107.17])/90*DEG90
Place_down_joint = np.array([-25.45,-56.45,89.79,-123.20,-89.68,107.37])/90*DEG90

assert (Target_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()


robot.execute_arm(init_joint)

robot.execute_arm(Target_up_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))

robot.execute_arm(Target_down_joint)

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

# 들고
robot.pmove_xc(np.array([0,2000,3000,1000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# # 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

robot.execute_arm(Target_up_joint)
robot.execute_arm(Place_up_joint)
robot.execute_arm(Place_down_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))
robot.execute_arm(Place_up_joint)
robot.execute_arm(init_joint)


Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan


# Tumbler2

In [46]:
robot.pmove_xc(np.array([0,0,0,0]*4))

In [47]:
init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
Target_up_joint = np.array([-11.76,-94.52,93.83,-89.17,-89.60,120.91])/90*DEG90
Target_down_joint = np.array([-11.76,-91.46,111.03,-109.48,-89.64,121.01])/90*DEG90
Place_up_joint = np.array([-8.21,-66.55,61.03,-84.34,-89.52,124.35])/90*DEG90
Place_down_joint = np.array([-8.26,-57.02,90.09,-122.93,-89.67,124.56])/90*DEG90

assert (Target_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()


robot.execute_arm(init_joint)

robot.execute_arm(Target_up_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))

robot.execute_arm(Target_down_joint)

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

# 들고
robot.pmove_xc(np.array([0,2000,3000,1000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

# 들고
robot.pmove_xc(np.array([0,2000,3000,1000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))


robot.execute_arm(Target_up_joint)
robot.execute_arm(Place_up_joint)
robot.execute_arm(Place_down_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))
robot.execute_arm(Place_up_joint)
robot.execute_arm(init_joint)


Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan


## Gat

In [48]:
robot.pmove_xc(np.array([0,0,0,0]*4))


In [49]:
init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
Target_up_joint   = np.array([16.84,-83.79,82.93,-89.01,-89.59,149.49])/90*DEG90
Target_down_joint = np.array([16.81,-82.13,96.38,-104.12,-89.61,149.57])/90*DEG90
Place_up_joint    = np.array([12.25,-55.41,42.79,-77.25,-89.58,144.78])/90*DEG90
Place_down_joint  = np.array([12.20,-48.98,77.21,-118.10,-89.63,145.01])/90*DEG90

assert (Target_up_joint * np.array([1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint * np.array([1,-1,1,-1,-1,1])>0).all()
assert (Place_up_joint * np.array([1,-1,1,-1,-1,1])>0).all()
assert (Place_down_joint * np.array([1,-1,1,-1,-1,1])>0).all()


robot.execute_arm(init_joint)

robot.execute_arm(Target_up_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))

robot.execute_arm(Target_down_joint)


# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1500,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,1200,1600]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))


robot.execute_arm(Target_up_joint)
robot.execute_arm(Place_up_joint)
robot.execute_arm(Place_down_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))
robot.execute_arm(Place_up_joint)
robot.execute_arm(init_joint)


Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan


In [50]:
robot.pmove_xc(np.array([0,0,0,0]*4))

# MAYO

In [17]:
pos = initpos+ PROPORTION_MATRIX @ np.array([1,0,1,0, 0,0,0,0, 0,0,0,0, 1,0,1,0]) * 4000 * np.array([1,-1,-1,1]*4)
pos = pos    + PROPORTION_MATRIX @ np.array([0,0,0,0, 0.3,0,1,0, 1,0,0,0.3, 0,0,0,0]) * 4000 * np.array([1,-1,-1,1]*4)

# pos = pos + np.array([0,0,0,0, 0,0,0,0, 0,0,0,400, 0,0,0,0]) * np.array([1,-1,-1,1]*4)
snapbot.set_goalposcluster(pos,1)

In [18]:
pos = initpos+ PROPORTION_MATRIX @ np.array([1,0,1,0, 0,0,0,0, 0,0,0,0, 1,0,1,0]) * 4000 * np.array([1,-1,-1,1]*4)
pos = pos    + PROPORTION_MATRIX @ np.array([0,0,0,0, 0.3,0,1,0, 1,0,0,0.3, 0,0,0,0]) * 4000 * np.array([1,-1,-1,1]*4)
pos = pos    + PROPORTION_MATRIX @ np.array([0,0,0,0, 0,0,1,0, 1,0,0,0, 0,0,0,0]) * 1000 * np.array([1,-1,-1,1]*4)

# pos = pos + np.array([0,0,0,0, 0,0,0,0, 0,0,0,400, 0,0,0,0]) * np.array([1,-1,-1,1]*4)
snapbot.set_goalposcluster(pos,1)